In [4]:
from gan_synthesis.u_net_models.unet import UNet

In [5]:
model = UNet()

In [6]:
import torch
from gan_synthesis.preprocessing.transforms import read_cropped


image = read_cropped(t="contrast", idx=123, as_type="pt").unsqueeze(0)
model.eval()
with torch.no_grad():
    recon = model(image)

torch.Size([1, 64, 24, 24]) torch.Size([1, 64, 24, 24])


AttributeError: 'NoneType' object has no attribute 'shape'

In [4]:
recon.shape

torch.Size([1, 4, 96, 96])